## Search Engine With Tools And Agents

In [ ]:
## Arxiv -- Research
## tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [2]:
## used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=100)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
## used the inbuilt tool of arxiv
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=100)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [4]:
tools = [wiki,arxiv]

In [5]:
### custom tools create [RAG TOOL]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
# creat a Embeddign by huggingface
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

e:\LC-basics[1]\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
loader = WebBaseLoader("https://python.langchain.com/docs/integrations/tools/")
docs=loader.load()
document = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectorDB=FAISS.from_documents(documents=document,embedding=embedding)
retriever =vectorDB.as_retriever()
retriever

e:\LC-basics[1]\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022B01C16E30>, search_kwargs={})

In [9]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langchain-tools", "search any information about langchain tools")

retriever_tool.name

'langchain-tools'

In [10]:
tools = [wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\LC-basics[1]\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=100)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=100)),
 Tool(name='langchain-tools', description='search any information about langchain tools', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000022B02C969E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022B01C16E30>, searc

In [ ]:
## Run all this tools with Agents and LLM Models

## Tools , LLM ----> AgentExecutor

from langchain_groq import ChatGroq
llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama3-70b-8192"
)

In [12]:
## crate prompts template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

In [13]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [16]:
##  Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [17]:
## Agent Excuter
from langchain.agents import AgentExecutor
agent_exicuter = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_exicuter

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [25]:
agent_exicuter.invoke({"input":"New vice president of india in 2025"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'vice president of india 2025'}`


Page: Vice President of India
Summary: The Vice President of India (ISO: Bhārata kē Uparāṣṭrapati) i
Invoking: `wikipedia` with `{'query': 'list of vice presidents of india'}`


Page: List of vice presidents of India
Summary: The vice president of India is the second highest coThe current vice president of India is Jagdeep Dhankhar, who took office on August 11, 2022. However, since you asked about the vice president in 2025, I couldn't find any information on who will be the vice president in 2025 as it's a future event.

> Finished chain.


{'input': 'New vice president of india in 2025',
 'output': "The current vice president of India is Jagdeep Dhankhar, who took office on August 11, 2022. However, since you asked about the vice president in 2025, I couldn't find any information on who will be the vice president in 2025 as it's a future event."}

In [24]:
agent_exicuter.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligenceMachine learning (ML) is a field of study in artificial intelligence that involves developing algorithms and statistical models that enable machines to learn from data, make decisions, and improve their performance on a given task over time.

> Finished chain.


{'input': 'What is machine learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence that involves developing algorithms and statistical models that enable machines to learn from data, make decisions, and improve their performance on a given task over time.'}

In [30]:
agent_exicuter.invoke({"input":"give me a link can i search online (ISO: Bhārata kē Pradhānamantrī)"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Prime Minister of India'}`


Page: Prime Minister of India
Summary: The prime minister of India (ISO: Bhārata kē Pradhānamantrī) 
Invoking: `wikipedia` with `{'query': 'List of Prime Ministers of India'}`


Page: List of prime ministers of India
Summary: The prime minister of India is the chief executive oIt seems like you're looking for a list of prime ministers of India. Here is the list:

1. Jawaharlal Nehru (1947-1964)
2. Lal Bahadur Shastri (1964-1966)
3. Indira Gandhi (1966-1977)
4. Morarji Desai (1977-1979)
5. Charan Singh (1979-1980)
6. Indira Gandhi (1980-1984)
7. Rajiv Gandhi (1984-1989)
8. V. P. Singh (1989-1990)
9. Chandra Shekhar (1990-1991)
10. P. V. Narasimha Rao (1991-1996)
11. H. D. Deve Gowda (1996-1997)
12. I. K. Gujral (1997-1998)
13. Atal Bihari Vajpayee (1998-2004)
14. Manmohan Singh (2004-2014)
15. Narendra Damodardas Modi (2014-present)

Let me know if you need anything else!

> Fini

{'input': 'give me a link can i search online (ISO: Bhārata kē Pradhānamantrī)',
 'output': "It seems like you're looking for a list of prime ministers of India. Here is the list:\n\n1. Jawaharlal Nehru (1947-1964)\n2. Lal Bahadur Shastri (1964-1966)\n3. Indira Gandhi (1966-1977)\n4. Morarji Desai (1977-1979)\n5. Charan Singh (1979-1980)\n6. Indira Gandhi (1980-1984)\n7. Rajiv Gandhi (1984-1989)\n8. V. P. Singh (1989-1990)\n9. Chandra Shekhar (1990-1991)\n10. P. V. Narasimha Rao (1991-1996)\n11. H. D. Deve Gowda (1996-1997)\n12. I. K. Gujral (1997-1998)\n13. Atal Bihari Vajpayee (1998-2004)\n14. Manmohan Singh (2004-2014)\n15. Narendra Damodardas Modi (2014-present)\n\nLet me know if you need anything else!"}

In [33]:
agent_exicuter.invoke({"input":"how am i learn gen ai in details"})



> Entering new AgentExecutor chain...

Invoking: `langchain-tools` with `{'query': 'learn gen ai in details'}`




e:\LC-basics[1]\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Webhooks.InfobipThis notebook that shows how to use Infobip API wrapper to send SMS m...Ionic Shopping ToolIonic is a plug and play ecommerce marketplace for AI Assistants. By ...JenkinsTools for interacting with Jenkins.Jina SearchThis notebook provides a quick overview for getting started with Jina...Jira ToolkitThis notebook goes over how to use the Jira toolkit.JSON ToolkitThis notebook showcases an agent interacting with large JSON/dict obj...Lemon AgentLemon Agent helps you build powerful AI assistants in minutes and aut...LinkupSearchToolLinkup provides an API to connect LLMs to the web and the Linkup Prem...MemgraphThis will help you get started with the Memgraph toolkit.MemorizeFine-tuning LLM itself to memorize information using unsupervised lea...Mojeek SearchThe following notebook will explain how to get results using Mojeek S...MultiOn ToolkitMultiON has built an AI Agent that can interact with a broad array of...NASA ToolkitThis notebook shows how to use agents to

notebo

e:\LC-basics[1]\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Invoking: `arxiv` with `{'query': 'gen ai details'}`
responded: It seems like you're looking for information on how to learn about Gen AI in detail. Based on the output from the previous tool calls, I'll try to provide a more specific response.

Since you're interested in learning about Gen AI, I'll suggest a more relevant tool call. Here it is:



Published: 2025-03-12
Title: Generative AI Policies under the Microscope: How CS Conferences Are NavIt seems like the tool call yielded a relevant result. Since the result is a research paper related to generative AI, I'll respond directly without using a tool. 

The paper appears to discuss the policies of generative AI in the context of computer science conferences. It's likely that the paper explores the implications of generative AI on the field of computer science and how conferences are navigating these changes.

If you're interested in learning more about generative AI, I can provide a brief overview. Generative AI refers to a type o

{'input': 'how am i learn gen ai in details',
 'output': "It seems like the tool call yielded a relevant result. Since the result is a research paper related to generative AI, I'll respond directly without using a tool. \n\nThe paper appears to discuss the policies of generative AI in the context of computer science conferences. It's likely that the paper explores the implications of generative AI on the field of computer science and how conferences are navigating these changes.\n\nIf you're interested in learning more about generative AI, I can provide a brief overview. Generative AI refers to a type of artificial intelligence that can generate new, original content, such as text, images, or music. This technology has the potential to revolutionize various industries, from entertainment to healthcare.\n\nWould you like me to explain more about generative AI or is there something specific you'd like to know?"}